In [ ]:
from sklearn.model_selection import StratifiedKFold
from lazyqsar.qsar import LazyBinaryQSAR
from lazyqsar.agnostic import LazyBinaryClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import lazyqsar
import h5py
import tqdm
import os

# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = "."

# Load data
data = pd.read_csv(os.path.join(root, "..", "output", "mtuberculosis", "datasets", "CHEMBL4649948_IC50_umol.L-1_perc_1.csv.gz"))
X = data['canonical_smiles'].astype(str).tolist()[:500]
Y = data['bin'].astype(np.int8).to_numpy()[:500]

# Define stratified 5 fold CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Load descriptors
with h5py.File(os.path.join(root, "..", "output", "mtuberculosis", "descriptors.h5"), "r") as f:
    SMILES = f['SMILES'][:]
    X_Morgan = f['X_Morgan'][:]

# Define dict mapping smiles to morgan fingerprints
SMILES_TO_MORGAN = {
    smiles.decode("utf-8"): fp
    for (chembl_id, smiles), fp in zip(SMILES, X_Morgan)}

# For each split
AUROCS = []
for train, test in skf.split(X, Y):
    print(len(train), len(test))

    # Get train/test indices
    X_train, Y_train = [X[i] for i in train], Y[train]
    X_test, Y_test = [X[i] for i in test], Y[test]

    # Get Morgan Fingerprints and train model
    X_train = np.array([SMILES_TO_MORGAN[smi] for smi in X_train])
    X_test  = np.array([SMILES_TO_MORGAN[smi] for smi in X_test])
    model = LazyBinaryClassifier(mode="fast")
    model.fit(X=X_train, y=Y_train)

    # Store results
    AUROCS.append(roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1]))
    break

# Train full model
X = np.array([SMILES_TO_MORGAN[smi] for smi in X])
model = LazyBinaryClassifier(mode="fast")
model.fit(X=X, y=Y)

# Save model
model.save(os.path.join(root, "..", "output", "mtuberculosis", "models", "CHEMBL4649948_IC50_umol.L-1_perc_1.zip"))

# Save CV results
with open(os.path.join(root, "..", "output", "mtuberculosis", "models", "CHEMBL4649948_IC50_umol.L-1_perc_1.csv"), "w") as outfile:
    outfile.write(",".join([str(round(i, 3)) for i in AUROCS]))

/home/acomajuncosa/miniconda3/envs/lazyqsar/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


400 100


14:40:02 INFO     Dataset summary:                                                                                 
                    • Total samples: 400                                                                           
                    • Positive samples: 19                                                                         
                    • Negative samples: 381                                                                        
                    • Max samples per partition: 10000                                                             
                    • Min samples per partition: 30                                                                
                    • Positive proportion: 0.05

         INFO     Original positive samples: 19, total samples: 400

         INFO     Maximum samples: 10000

...

         INFO         Output: output, shape: ['batch_size']

         INFO     ****************************

         INFO     Cleaning up intermediate files...

14:42:55 INFO     Copied partition ONNX model to                                                                   
                  ./../output/mtuberculosis/models/CHEMBL4649948_IC50_umol.L-1_perc_1/model_000.onnx

'./../output/mtuberculosis/models/CHEMBL4649948_IC50_umol.L-1_perc_1.zip'